In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv2D, MaxPooling2D, Flatten, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv('input9.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

WITHOUT_CLASSIFICATION    58204
DESIGN                     2703
IMPLEMENTATION              757
DEFECT                      472
TEST                         85
DOCUMENTATION                54
Name: class, dtype: int64

In [5]:
df_satd = df[(df['class']=='DESIGN') | (df['class']== 'IMPLEMENTATION' ) | (df['class']== 'DEFECT') | (df['class']== 'TEST') | (df['class']== 'DOCUMENTATION') ]


In [6]:
df_nsatd = df[df['class']=='WITHOUT_CLASSIFICATION']

In [7]:

df_nsatd_downsampled = df_nsatd.sample(df_satd.shape[0])

In [8]:
df_balanced = pd.concat([df_nsatd_downsampled, df_satd])

In [9]:
df_balanced['class'].value_counts()

WITHOUT_CLASSIFICATION    4071
DESIGN                    2703
IMPLEMENTATION             757
DEFECT                     472
TEST                        85
DOCUMENTATION               54
Name: class, dtype: int64

In [10]:
df_balanced['spam']=df_balanced['class'].apply(lambda x: 0 if x=='WITHOUT_CLASSIFICATION' else 1)

In [11]:
df_balanced['spam'].value_counts()

0    4071
1    4071
Name: spam, dtype: int64

In [12]:
df_balanced.head()

,comment,class,spam
24079,non Javadoc \t @see org columba mail folder...,WITHOUT_CLASSIFICATION,0
58346,i18n graph addChildTables Add child tables,WITHOUT_CLASSIFICATION,0
4517,check if filename defined,WITHOUT_CLASSIFICATION,0
5860,$NON NLS 1$ Ensure variable has been provided,WITHOUT_CLASSIFICATION,0
8664,Prevent instantiation,WITHOUT_CLASSIFICATION,0


In [13]:
df_balanced['classes']=df_balanced['spam'].apply(lambda x: "Classified" if x==1 else "Not-Classified")

In [14]:
df_balanced.head()

,comment,class,spam,classes
24079,non Javadoc \t @see org columba mail folder...,WITHOUT_CLASSIFICATION,0,Not-Classified
58346,i18n graph addChildTables Add child tables,WITHOUT_CLASSIFICATION,0,Not-Classified
4517,check if filename defined,WITHOUT_CLASSIFICATION,0,Not-Classified
5860,$NON NLS 1$ Ensure variable has been provided,WITHOUT_CLASSIFICATION,0,Not-Classified
8664,Prevent instantiation,WITHOUT_CLASSIFICATION,0,Not-Classified


In [15]:
X = df_balanced['comment']
Y = df_balanced['spam']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify=Y, random_state=42)

max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [17]:
print(len(train_sequences_matrix[0]))

250


In [18]:
def CNN():
    cnn = tf.keras.Sequential()
    cnn.add(Input(name='inputs',shape=[max_len]))
    cnn.add(Embedding(max_words,50,input_length=max_len))
    cnn.add(Conv1D(128, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(256, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(512, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))

    cnn.add(Flatten())
    cnn.add(Dense(256, activation ='relu'))
    cnn.add(Dense(64, activation ='relu'))
    cnn.add(Dense(1, activation='sigmoid'))

    return cnn

In [19]:
model = CNN()
model.summary()
opt = tf.keras.optimizers.Adadelta(learning_rate=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 50)           500000    
                                                                 
 conv1d (Conv1D)             (None, 250, 128)          19328     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 125, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 125, 256)          98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 62, 256)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 62, 512)           3

In [20]:
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(train_sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.25, callbacks=[model_checkpoint_callback,reduce_lr])

model.load_weights(checkpoint_filepath)

Epoch 1/50
39/39 [==============================] - 5s 40ms/step - loss: 0.6918 - accuracy: 0.5274 - val_loss: 0.6912 - val_accuracy: 0.6642 - lr: 0.1000
Epoch 2/50
39/39 [==============================] - 1s 28ms/step - loss: 0.6889 - accuracy: 0.6323 - val_loss: 0.6873 - val_accuracy: 0.6679 - lr: 0.1000
Epoch 3/50
39/39 [==============================] - 1s 27ms/step - loss: 0.6828 - accuracy: 0.6550 - val_loss: 0.6782 - val_accuracy: 0.6458 - lr: 0.1000
Epoch 4/50
39/39 [==============================] - 1s 28ms/step - loss: 0.6670 - accuracy: 0.6665 - val_loss: 0.6571 - val_accuracy: 0.6366 - lr: 0.1000
Epoch 5/50
39/39 [==============================] - 1s 28ms/step - loss: 0.6233 - accuracy: 0.6902 - val_loss: 0.6059 - val_accuracy: 0.6618 - lr: 0.1000
Epoch 6/50
39/39 [==============================] - 1s 28ms/step - loss: 0.5655 - accuracy: 0.7160 - val_loss: 0.5475 - val_accuracy: 0.7452 - lr: 0.1000
Epoch 7/50
39/39 [==============================] - 1s 28ms/step - loss: 0.5

In [21]:
model.save("CNN_balanced.h5")

In [22]:
accr = model.evaluate(test_sequences_matrix,Y_test)

51/51 [==============================] - 0s 7ms/step - loss: 0.1960 - accuracy: 0.9319


In [23]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.196
  Accuracy: 0.932


In [24]:
y_pred = model.predict(test_sequences_matrix)

51/51 [==============================] - 0s 6ms/step


In [25]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [26]:
print(classification_report(Y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.930318  0.933742  0.932027       815
           1   0.933416  0.929975  0.931692       814

    accuracy                       0.931860      1629
   macro avg   0.931867  0.931859  0.931860      1629
weighted avg   0.931866  0.931860  0.931860      1629

